# Подготовка смержинных текстов ms-marco

In [1]:
!pip install import-ipynb
%cd '/content/drive/My Drive/QA_training/notebooks'
import import_ipynb
from my_functions import *

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=97e0c75ce1c37a559844aa6fed53d6eadfd5d0b5e0fb6cb7a1aee4ce4d3d8bbc
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb
/content/drive/My Drive/QA_training/notebooks
importing Jupyter notebook from my_functions.ipynb
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-b2ooec2h
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-b2ooec2h
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 890kB 4.3MB/s 
     |████████████████████████████████| 2.9MB 33.9MB/s 
  Created wheel for transformers: filename=transformers-4.2.0.dev0-cp36-none-any.whl size=1680279 sha256=a29b61d06e9d9ab00600865133f4e8a9949766d3337daed9b339585caced08ce
  Store

# Соединение последовательных текстов:

Импорт ms-marco датасета:

In [2]:
!pip install datasets
from datasets import load_dataset
dataset = load_dataset(
   'ms_marco', 'v2.1')

Dataset ms_marco downloaded and prepared to /root/.cache/huggingface/datasets/ms_marco/v2.1/2.1.0/8378931e642240518368077ec1cc5b794130258f94ed47a957aba95e8910912a. Subsequent calls will reuse this data.


В датасете, который дал Дмитрий, находятся только часть QAT триплетов из оригинального датасета. Нужно найти id этих триплетов, чтобы подобрать вопросы из тойже темы.

In [4]:
def return_idx(original_questions, required_questions):
    indexes = []
    k = 1
    for question in pb(required_questions):
      id = original_questions.index(question)
      indexes.append(id)
      if k % 1000 == 0:
        original_questions = original_questions[id:]
      k += 1
    return indexes


In [3]:
datasets = [pd.read_csv('/content/drive/MyDrive/QA_training/datasets_csv/ms_initial_train.csv'),
            pd.read_csv('/content/drive/MyDrive/QA_training/datasets_csv/ms_initial_dev.csv')]
data_types = ['train', 'validation']

In [7]:
original_questions = [dataset['train']['query'], 
                      dataset['validation']['query']]
required_questions = [dataset.question for dataset in datasets]

In [8]:
indexes = []

for i in range(2):
    indexes.append(return_idx(original_questions[i], required_questions[i]))

100% (477580 of 477580) |################| Elapsed Time: 0:00:11 Time:  0:00:11
100% (52442 of 52442) |##################| Elapsed Time: 0:00:00 Time:  0:00:00


Теперь надо добавить дополнительные тексты, коотрые относятся к вопросу по теме, но не содеражат ответ.

In [44]:
def add_texts(idx, original_data, num):
    texts = []

    for id in pb(idx):
        if num == 1:
            text_num = original_data[id]['passages']['is_selected'].index(0)
            texts.append(original_data[id]['passages']['passage_text'][text_num])
        else:
            string = []
            start = 0
            while len(string) < num:
                index = original_data[id]['passages']['is_selected'].index(0, start)
                new_text = original_data[id]['passages']['passage_text'][index]
                start = index+1
                if new_text not in string:
                    string.append(new_text)
                
            texts.append(string)
    return texts


In [45]:
additional_texts = []
for i in range(2):
    additional_texts.append(add_texts(indexes[i], dataset[data_types[i]], 2))

100% (477580 of 477580) |################| Elapsed Time: 0:03:26 Time:  0:03:26
100% (52442 of 52442) |##################| Elapsed Time: 0:00:22 Time:  0:00:22


Теперь добавляем новые тексты к оригинальным и перемешываем все, чтобы в настоящий текст, содержащий ответ, стоял на рандомном месте.

In [31]:
def prepare_dataset(df, additional_texts):
    l = len(additional_texts[0])
    k = 0
    data = []

    for index, row in pb(df.iterrows()):
        if l == 1:
            texts = [row['text'], additional_texts[k]]
        else:
            texts = [row['text'], *additional_texts[k]]

        np.random.shuffle(texts)
        data.append([row['question'], *texts, row['span_answer'], row['answer']])
        k += 1
    texts_names = [f"t{i+1}" for i in range(l+1)]
    
    return pd.DataFrame(data=data, columns = ['question', *texts_names, 'span_answer', 'answer'])

In [46]:
new_dfs = []
for i in range(2):
    new_dfs.append(prepare_dataset(datasets[i], additional_texts[i]))

| |                #                             | 477579 Elapsed Time: 0:01:02
| |                            #                  | 52441 Elapsed Time: 0:00:06


In [47]:
new_dfs[1].head()

,question,t1,t2,t3,span_answer,answer
0,. what is a corporation?,"Today, there is a growing community of more th...",McDonald's Corporation is one of the most reco...,A company is incorporated in a specific nation...,A corporation is a company or group of people ...,A corporation is a company or group of people ...
1,symptoms of a dying mouse,This can be fatal quite quickly to mice. 1 It...,The symptoms are similar but the mouse will be...,The symptoms of mites include: 1 excessive sc...,the mouse will be in much worse condition: run...,"The symptoms of a dying mouse are runny eyes, ..."
2,average number of lightning strikes per day,Although many lightning flashes are simply clo...,Quick Answer. Lightning strikes reach the grou...,Lightning is a major cause of storm related de...,"Globally, 8,640,000 lightning strikes per","Globally 8,640,000 lightning strikes per day."
3,can you burn your lawn with fertilizer,These spots are generally caused by a buildup ...,Fertilizer burn is the result of over fertiliz...,Verify the reason for the lawn burn. Grass wil...,over fertilizing,"Yes, over fertilizing can burn lawn."
4,goch meaning,"See the Goch surname, family crest & coat of a...",The name Goch is derived from the Welsh word c...,The name Goch is derived from the Welsh word c...,The name Goch is,The meaning of the name Goch is red.


In [48]:
path = '/content/drive/MyDrive/QA_training/merged datasets/ms_marco'
new_dfs[0].to_csv(path+'/merged_by_3_train.csv', index=False)
new_dfs[1].to_csv(path+'/merged_by_3_dev.csv', index=False)

# Соединение рандомных текстов

Для того, чтобы в каждом триплете к оригинальному тексту добавить рандомные тексты, тостаточно простом перемешать дополнительные тексты, которые были получены ранее


In [53]:
texts_copy = additional_texts.copy()

In [54]:
for text in texts_copy:
    np.random.shuffle(text)

In [55]:
new_random_dfs = []
for i in range(2):
    new_random_dfs.append(prepare_dataset(datasets[i], texts_copy[i]))

| |                     #                        | 477579 Elapsed Time: 0:01:02
| |                        #                      | 52441 Elapsed Time: 0:00:06


In [57]:
new_random_dfs[1].head()

,question,t1,t2,t3,span_answer,answer
0,. what is a corporation?,McDonald's Corporation is one of the most reco...,Consider these designs when deciding how to gr...,Defined as plants that retain their leaves yea...,A corporation is a company or group of people ...,A corporation is a company or group of people ...
1,symptoms of a dying mouse,The only difference between RPKM and FPKM is t...,Here’s how you calculate TPM: 1 Divide the re...,The symptoms are similar but the mouse will be...,the mouse will be in much worse condition: run...,"The symptoms of a dying mouse are runny eyes, ..."
2,average number of lightning strikes per day,"What do the income effect, the substitution ef...",Increasing marginal costs can be identified us...,Although many lightning flashes are simply clo...,"Globally, 8,640,000 lightning strikes per","Globally 8,640,000 lightning strikes per day."
3,can you burn your lawn with fertilizer,The style of these buildings evinces that the ...,Fertilizer burn is the result of over fertiliz...,ignorant - unaware because of a lack of releva...,over fertilizing,"Yes, over fertilizing can burn lawn."
4,goch meaning,The name Goch is derived from the Welsh word c...,Union painters usually earn by the hour and ca...,"The national average Painter salary is $31,200...",The name Goch is,The meaning of the name Goch is red.


In [58]:
path = '/content/drive/MyDrive/QA_training/merged datasets/ms_marco'
new_random_dfs[0].to_csv(path+'/random_merged_by_3_train.csv', index=False)
new_random_dfs[1].to_csv(path+'/random_merged_by_3_dev.csv', index=False)

# Проверим, есть ли одинаковые тексты



In [49]:
path = '/content/drive/MyDrive/QA_training/merged datasets/ms_marco'
dev_df = pd.read_csv(path+'/merged_by_3_train.csv')

In [50]:
dev_df

,question,t1,t2,t3,span_answer,answer
0,)what was the immediate impact of the success ...,The Manhattan Project and its atomic bomb help...,The presence of communication amid scientific ...,Essay on The Manhattan Project - The Manhattan...,the success of the Manhattan Project as scient...,The immediate impact of the success of the man...
1,_________ justice is designed to repair the ha...,punishment designed to repair the damage done ...,"group discussions, community boards or panels ...",The approach is based on a theory of justice t...,Restorative justice that fosters dialogue betw...,Restorative justice that fosters dialogue betw...
2,why did stalin want control of eastern europe,"The Tuvan People's Republic, was proclaimed in...",Western betrayal. The concept of Western betra...,There are 3 main reasons why Stalin wanted to ...,reasons why Stalin wanted to control Eastern E...,The reasons why Stalin wanted to control Easte...
3,why do nails get rusty,A: Nails rust in water because water allows th...,Just asked! See more. 1 What is the minimum c...,what to Do If I Stepped on Rusty Nail: Prevent...,Nails rust in water because water allows the i...,Nails rust in water because water allows the i...
4,depona ab,A preview of what LinkedIn members have to say...,"Depona Ab is a library in Vilhelmina, Sweden. ...",Depona Oy YTJ: Y-tunnus: 21527338 YTJ: Toimial...,Depona Ab is a library in,"Depona Ab is a library in Vilhelmina, Sweden."
...,...,...,...,...,...,...
477575,fort selden roofing las cruces phone number,Traditional IRA. A traditional tax-deferred IR...,Print | Save | Directions. Fort Selden Roofing...,This deduction is equal to the actual amount o...,Print | Save | Directions. Fort Selden Roofing...,(575) 527-1709
477576,nasa credit union routing number,The electric dipole moment for a pair of oppos...,Dipole moment may refer to: 1 Electric dipole...,Check NASA FEDERAL CREDIT UNION routing number...,NaN,NaN
477577,what county is hitchcock texas located in,Definition of modern for English Language Lear...,meaning are viewed as imperialistic efforts to...,The City of Hitchcock is located within La Mar...,Galveston,Galveston County
477578,what is blowout for hair,"Let’s face it, bad hair days blow. Thankfully ...",Females may be included in the Selective Servi...,"In doing so, it may force Congress or the pres...",all about great blowout style for about thirty...,All about great blowout style for about thirty...


In [51]:
texts = dev_df[['t1', 't2', 't3']].to_numpy()

In [52]:
n = 0
for t1, t2, t3 in texts:
  if t1 == t2 or t2==t3 or t1==t3:
    n += 1
print(n)

0


Совпадений текстов нет

### Функция, которая подготавливает тренировочный датасет: Разбивает QAT тройку с несколькими текстами на несколько троек с одиночными текстами

In [ ]:
df = pd.read_csv(path+'/merged_by_2_train.csv')
df

,question,t1,t2,span_answer,answer
0,)what was the immediate impact of the success ...,The presence of communication amid scientific ...,The Manhattan Project and its atomic bomb help...,the success of the Manhattan Project as scient...,The immediate impact of the success of the man...
1,_________ justice is designed to repair the ha...,The approach is based on a theory of justice t...,"group discussions, community boards or panels ...",Restorative justice that fosters dialogue betw...,Restorative justice that fosters dialogue betw...
2,why did stalin want control of eastern europe,There are 3 main reasons why Stalin wanted to ...,Western betrayal. The concept of Western betra...,reasons why Stalin wanted to control Eastern E...,The reasons why Stalin wanted to control Easte...
3,why do nails get rusty,A: Nails rust in water because water allows th...,what to Do If I Stepped on Rusty Nail: Prevent...,Nails rust in water because water allows the i...,Nails rust in water because water allows the i...
4,depona ab,A preview of what LinkedIn members have to say...,"Depona Ab is a library in Vilhelmina, Sweden. ...",Depona Ab is a library in,"Depona Ab is a library in Vilhelmina, Sweden."
...,...,...,...,...,...
477575,fort selden roofing las cruces phone number,Print | Save | Directions. Fort Selden Roofing...,Traditional IRA. A traditional tax-deferred IR...,Print | Save | Directions. Fort Selden Roofing...,(575) 527-1709
477576,nasa credit union routing number,The electric dipole moment for a pair of oppos...,Check NASA FEDERAL CREDIT UNION routing number...,NaN,NaN
477577,what county is hitchcock texas located in,Definition of modern for English Language Lear...,The City of Hitchcock is located within La Mar...,Galveston,Galveston County
477578,what is blowout for hair,Females may be included in the Selective Servi...,"Let’s face it, bad hair days blow. Thankfully ...",all about great blowout style for about thirty...,All about great blowout style for about thirty...


In [ ]:
def prepare_training_data(dataset, text_number):
    unpacked_data = []
    for index, row in dataset.iterrows():
        text_columns = [f"t{i+1}" for i in range(text_number)]
        answer = row['span_answer']
        for text_name in text_columns:
            if type(answer) == str and answer in row[text_name]:
                unpacked_data.append([row['question'], row[text_name], row['span_answer'], row['answer']])
            else:
              unpacked_data.append([row['question'], row[text_name], float('nan'), float('nan')])
    df = pd.DataFrame(data = unpacked_data, columns=['question', 'text', 'span_answer', 'answer'])
    return df

In [ ]:
new_df = prepare_training_data(df, 2)
new_df

,question,text,span_answer,answer
0,)what was the immediate impact of the success ...,The presence of communication amid scientific ...,the success of the Manhattan Project as scient...,The immediate impact of the success of the man...
1,)what was the immediate impact of the success ...,The Manhattan Project and its atomic bomb help...,NaN,NaN
2,_________ justice is designed to repair the ha...,The approach is based on a theory of justice t...,Restorative justice that fosters dialogue betw...,Restorative justice that fosters dialogue betw...
3,_________ justice is designed to repair the ha...,"group discussions, community boards or panels ...",NaN,NaN
4,why did stalin want control of eastern europe,There are 3 main reasons why Stalin wanted to ...,reasons why Stalin wanted to control Eastern E...,The reasons why Stalin wanted to control Easte...
...,...,...,...,...
955155,what county is hitchcock texas located in,The City of Hitchcock is located within La Mar...,Galveston,Galveston County
955156,what is blowout for hair,Females may be included in the Selective Servi...,NaN,NaN
955157,what is blowout for hair,"Let’s face it, bad hair days blow. Thankfully ...",all about great blowout style for about thirty...,All about great blowout style for about thirty...
955158,name some organisms that might live in a marin...,Alluvial diamond mining is the term used to de...,NaN,NaN


In [ ]:
new_df.to_csv(path+'/unpacked_merged_by_2_train.csv', index=False)